<a href="https://colab.research.google.com/gist/leoyang429/bdba9312049f5d1b940f9cd8ab4c5211/cs446-final-project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
from google.colab import drive
from matplotlib import pyplot as plt
from google.colab import auth

import os
import numpy as np  
import re, time
import tensorflow as tf
import random
import math
from tensorflow import keras

import argparse
import pandas as pd

from glob import glob
from tqdm import tqdm
from keras import backend as K

import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv3D, MaxPool3D, Flatten, UpSampling3D, BatchNormalization
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.utils import to_categorical

# Mount to Google Drive

In [2]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Enable GPU

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

# 3D UNET-like CNN

In [19]:
img_type_num = 4
seg_class_num = 4
h_padded = 160 #150
w_padded = 208 #194
d_padded = 176 #164

smooth = 0.000001

max_epoch = 100
train_num = 204
batch_size = 2
dataset = np.load('/content/drive/My Drive/Colab Notebooks/data_pub.zip')
entries = list(dataset)
entries.sort()
print(entries[410])

validation/001_imgs


In [5]:
def dice_coefficient(y_true, y_pred):
  dc = []
  for i in range(seg_class_num):
    y_true_f = K.flatten(y_true[:,:,:,:,i])
    y_pred_f = K.flatten(y_pred[:,:,:,:,i])
    intersection = K.sum(y_true_f * y_pred_f)

    dc.append((2.0 * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))

  return tf.add_n(dc)/seg_class_num

def dice_coefficient_loss(y_true, y_pred):
    return 1-dice_coefficient(y_true, y_pred)

In [6]:
def unet(input_size = (h_padded, w_padded, d_padded, img_type_num)):
  inputs = Input(input_size)

  bn = BatchNormalization()(inputs)
  conv1 = Conv3D(8, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(bn)
  pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
  conv2 = Conv3D(32, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  conv2 = Conv3D(32, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(conv2)


  pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)
  conv3 = Conv3D(128, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  conv3 = Conv3D(128, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(conv3)


  pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)
  conv4 = Conv3D(256, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  conv4 = Conv3D(256, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(conv4)

  drop4 = Dropout(0.2)(conv4)

  pool4 = MaxPooling3D(pool_size=(2, 2, 2))(drop4)
  conv5 = Conv3D(384, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5 = Conv3D(384, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(conv5)

  drop5 = Dropout(0.2)(conv5)

  up6 = Conv3D(256, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 4)
  conv6 = Conv3D(256, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv3D(256, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(conv6)


  up7 = Conv3D(128, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 4)
  conv7 = Conv3D(128, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv3D(128, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(conv7)


  up8 = Conv3D(32, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 4)
  conv8 = Conv3D(32, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv3D(32, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(conv8)


  up9 = Conv3D(8, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 4)
  conv10 = Conv3D(8, 3, activation = 'tanh', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv11 = Conv3D(4, 3, activation = 'sigmoid', padding = 'same', kernel_initializer = 'he_normal')(conv10)
  

  model = Model(inputs = inputs, outputs = conv11)
  model.compile("adam", loss = dice_coefficient_loss, metrics=[dice_coefficient])

  return model

In [7]:
def train_gen():
  for epoch in range(max_epoch):
    train_img = np.zeros((batch_size, h_padded, w_padded, d_padded, img_type_num))
    train_seg = np.zeros((batch_size, h_padded, w_padded, d_padded, seg_class_num))
    sampler = list(range(0,204))
    random.shuffle(sampler)
    cnt = 0

    batch_num = math.floor(train_num/batch_size)
    for k in range(batch_num):
      for i, j in zip(sampler[k*batch_size:(k+1)*batch_size], range(batch_size)):
          cnt+=1
          img = dataset[entries[2*i+1]]
          img = np.moveaxis(img, 0, -1)
          seg = dataset[entries[2*i+2]]

          img, seg = padding(img, seg)
          seg_onehot = seg_to_onehot(seg)

          train_img[j] = img
          train_seg[j] = seg_onehot
      yield(train_img, train_seg)
      
    remaining_num = train_num - batch_num * batch_size
    if remaining_num > 0:
      train_img = np.zeros((remaining_num, h_padded, w_padded, d_padded, img_type_num))
      train_seg = np.zeros((remaining_num, h_padded, w_padded, d_padded, seg_class_num))
      for i, j in zip(sampler[batch_num * batch_size:], range(remaining_num)):
          cnt+=1
          img = dataset[entries[2*i+1]]
          img = np.moveaxis(img, 0, -1)
          seg = dataset[entries[2*i+2]]

          img, seg = padding(img, seg)

          seg_onehot = seg_to_onehot(seg)

          train_img[j] = img
          train_seg[j] = seg_onehot
      yield(train_img, train_seg)
    #print(cnt)
    K.set_value(model.optimizer.learning_rate, 0.001*pow(0.85,epoch+1))

In [8]:
def seg_to_onehot(seg):
  seg_onehot = to_categorical(seg.astype(int), num_classes=seg_class_num)
  return seg_onehot

In [9]:
def padding(img, seg):
  h_raw = img.shape[0]
  w_raw = img.shape[1]
  d_raw = img.shape[2]


  h_pad = (int((h_padded - h_raw)/2), int((h_padded - h_raw + 1)/2))
  w_pad = (int((w_padded - w_raw)/2), int((w_padded - w_raw + 1)/2))
  d_pad = (int((d_padded - d_raw)/2), int((d_padded - d_raw + 1)/2))

  img = np.pad(img, pad_width=(h_pad, w_pad, d_pad, (0, 0)), mode='constant', constant_values=0)
  
  seg = np.pad(seg, pad_width=(h_pad, w_pad, d_pad), mode='constant', constant_values=0)

  return img, seg

In [10]:
with tf.device('/device:GPU:0'):
    model = unet()

model.fit(train_gen(), steps_per_epoch = train_num / batch_size, epochs=max_epoch)

model.save('/content/drive/My Drive/Colab Notebooks/model/unet_dice_gpu')

Epoch 1/100
102/102 [==============================] - 186s 2s/step - loss: 0.6469 - dice_coefficient: 0.3531
Epoch 2/100
102/102 [==============================] - 159s 2s/step - loss: 0.4055 - dice_coefficient: 0.5945
Epoch 3/100
102/102 [==============================] - 159s 2s/step - loss: 0.3258 - dice_coefficient: 0.6742
Epoch 4/100
102/102 [==============================] - 159s 2s/step - loss: 0.3231 - dice_coefficient: 0.6769
Epoch 5/100
102/102 [==============================] - 159s 2s/step - loss: 0.3127 - dice_coefficient: 0.6873
Epoch 6/100
102/102 [==============================] - 159s 2s/step - loss: 0.2870 - dice_coefficient: 0.7130
Epoch 7/100
102/102 [==============================] - 159s 2s/step - loss: 0.2923 - dice_coefficient: 0.7077
Epoch 8/100
102/102 [==============================] - 159s 2s/step - loss: 0.2805 - dice_coefficient: 0.7195
Epoch 9/100
102/102 [==============================] - 159s 2s/step - loss: 0.2826 - dice_coefficient: 0.7174
Epoch 10/1

KeyboardInterrupt: ignored

# Test on Validation Set

In [21]:
# with tf.device('/device:GPU:0'):
#   model = keras.models.load_model('/content/drive/My Drive/Colab Notebooks/model/unet_dice_gpu')

validate_num = 68
validate_batch_num = 1

validate_img = np.zeros((validate_batch_num, h_padded, w_padded, d_padded, img_type_num))
validate_seg = []

batchnum = math.floor(validate_num/validate_batch_num)

total_dc = 0.0

for k in range(batchnum):
  #print('batch', k)
  for i in range(validate_batch_num):
      img = dataset[entries[2*validate_batch_num*k+2*i+410]]
      img = np.moveaxis(img, 0, -1)
      seg = dataset[entries[2*validate_batch_num*k+2*i+411]]
      
      shape = img.shape
  
      h_raw = shape[0]
      w_raw = shape[1]
      d_raw = shape[2]

      h_pad = (int((h_padded - h_raw)/2), int((h_padded - h_raw + 1)/2))
      w_pad = (int((w_padded - w_raw)/2), int((w_padded - w_raw + 1)/2))
      d_pad = (int((d_padded - d_raw)/2), int((d_padded - d_raw + 1)/2))

      seg = np.pad(seg, pad_width=(h_pad, w_pad, d_pad), mode='constant', constant_values=0)
      actu_seg = seg[h_pad[0]:h_raw+h_pad[0], w_pad[0]:w_raw+w_pad[0], d_pad[0]:d_raw+d_pad[0]]
      actu_seg = seg_to_onehot(actu_seg)
      actu_seg = tf.expand_dims(actu_seg,0)

      img = np.pad(img, pad_width=(h_pad, w_pad, d_pad, (0, 0)), mode='constant', constant_values=0)

      img = tf.expand_dims(img, 0)

      seg_hat = model.predict(img)

      actu_seg_hat = seg_hat[0,h_pad[0]:h_raw+h_pad[0], w_pad[0]:w_raw+w_pad[0], d_pad[0]:d_raw+d_pad[0]]
      actu_seg_hat = tf.expand_dims(actu_seg_hat,0)

      dc = dice_coefficient(actu_seg, actu_seg_hat)
      total_dc += dc

print("Average Dice Coefficient on Validation Set:", total_dc/validate_num)

Average Dice Coefficient on Validation Set: tf.Tensor(0.66537577, shape=(), dtype=float32)


# Testing & Encoding

In [22]:
def rlencode(x, dropna=False):
    """
    Run length encoding.
    Based on http://stackoverflow.com/a/32681075, which is based on the rle 
    function from R.
    
    Parameters
    ----------
    x : 1D array_like
        Input array to encode
    dropna: bool, optional
        Drop all runs of NaNs.
    
    Returns
    -------
    start positions, run lengths, run values
    
    """
    where = np.flatnonzero
    x = np.asarray(x)
    n = len(x)
    if n == 0:
                return (np.array([], dtype=int), 
                        np.array([], dtype=int), 
                        np.array([], dtype=x.dtype))

    starts = np.r_[0, where(~np.isclose(x[1:], x[:-1], equal_nan=True)) + 1]
    lengths = np.diff(np.r_[starts, n])
    values = x[starts]
    
    if dropna:
        mask = ~np.isnan(values)
        starts, lengths, values = starts[mask], lengths[mask], values[mask]
    
    return starts, lengths, values

In [23]:
# with tf.device('/device:GPU:0'):
#   model = keras.models.load_model('/content/drive/My Drive/Colab Notebooks/model/unet_dice_gpu')

dataset = np.load('/content/drive/My Drive/Colab Notebooks/test_pub.zip')
entries = list(dataset)
entries.sort()
print(entries)

test_case_num = 68

for i in range(test_case_num):
  img = dataset[entries[i+1]]
  img = np.moveaxis(img, 0, -1)
  #print(i)

  shape = img.shape
  
  h_raw = shape[0]
  w_raw = shape[1]
  d_raw = shape[2]

  h_pad = (int((h_padded - h_raw)/2), int((h_padded - h_raw + 1)/2))
  w_pad = (int((w_padded - w_raw)/2), int((w_padded - w_raw + 1)/2))
  d_pad = (int((d_padded - d_raw)/2), int((d_padded - d_raw + 1)/2))

  img = np.pad(img, pad_width=(h_pad, w_pad, d_pad, (0, 0)), mode='constant', constant_values=0)

  img = tf.expand_dims(img, 0)

  seg = model.predict(img)

  actu_seg = np.argmax(seg[0,h_pad[0]:h_raw+h_pad[0], w_pad[0]:w_raw+w_pad[0], d_pad[0]:d_raw+d_pad[0]], axis=3)
  #print(actu_seg[70, 70])
  np.save("/content/drive/My Drive/Colab Notebooks/test_seg/0"+str((i+1)//10)+str((i+1)%10)+"_seg.npy", actu_seg)

['test_pub/', 'test_pub/001_imgs', 'test_pub/002_imgs', 'test_pub/003_imgs', 'test_pub/004_imgs', 'test_pub/005_imgs', 'test_pub/006_imgs', 'test_pub/007_imgs', 'test_pub/008_imgs', 'test_pub/009_imgs', 'test_pub/010_imgs', 'test_pub/011_imgs', 'test_pub/012_imgs', 'test_pub/013_imgs', 'test_pub/014_imgs', 'test_pub/015_imgs', 'test_pub/016_imgs', 'test_pub/017_imgs', 'test_pub/018_imgs', 'test_pub/019_imgs', 'test_pub/020_imgs', 'test_pub/021_imgs', 'test_pub/022_imgs', 'test_pub/023_imgs', 'test_pub/024_imgs', 'test_pub/025_imgs', 'test_pub/026_imgs', 'test_pub/027_imgs', 'test_pub/028_imgs', 'test_pub/029_imgs', 'test_pub/030_imgs', 'test_pub/031_imgs', 'test_pub/032_imgs', 'test_pub/033_imgs', 'test_pub/034_imgs', 'test_pub/035_imgs', 'test_pub/036_imgs', 'test_pub/037_imgs', 'test_pub/038_imgs', 'test_pub/039_imgs', 'test_pub/040_imgs', 'test_pub/041_imgs', 'test_pub/042_imgs', 'test_pub/043_imgs', 'test_pub/044_imgs', 'test_pub/045_imgs', 'test_pub/046_imgs', 'test_pub/047_imgs',

In [24]:
# get all the npy files in folder
SEP = os.path.sep
files = glob("/content/drive/My Drive/Colab Notebooks/test_seg/*")

ids = []    # ImageId_ClassId columns
codes = []  # EncodedPixels columns

# iterate through the entire folders. Compute the run-length encoding for each segmenation npy file
for i, file in enumerate(tqdm(files)):
    # check if the file is seg (exclude img npy file)
    # npy file name are expected to have the format of XXX_seg.npy
    if not file.split(SEP)[-1][-7:-4] == 'seg':
        continue

    # im_name are expected to have the format of XXX_seg.npy
    im_name = file.split(SEP)[-1][:3]
    sample = np.load(file).flatten() # flatten the numpy array to 1d
    
    # get the start position, 
    # length for each segment correspoding to start position, 
    # and the value/labels for each segments
    starts, lengths, values = rlencode(sample)

    # these will be the encoded pixels strings for each image, 4 labels for each iamge
    code_1 = ''
    code_2 = ''
    code_3 = ''
    code_4 = ''
    # convert the run-length encoding output from rlencode function to the required csv format
    for start, length, value in zip(starts, lengths, values):
        if value == 0:
            code_1 += '{} {} '.format(start, length)
        elif value == 1:
            code_2 += '{} {} '.format(start, length)
        elif value == 2:
            code_3 += '{} {} '.format(start, length)
        elif value == 3:
            code_4 += '{} {} '.format(start, length)

    # update final output list, add for rows for each image (4 labels)
    for i in range(4):
        id = '{}_{}'.format(im_name, i)
        ids.append(id)
    codes.extend([code_1.strip(), code_2.strip(), code_3.strip(), code_4.strip()])


dic = {'ImageId_ClassId': ids, 'EncodedPixels': codes}
df = pd.DataFrame(dic)
df.to_csv("/content/drive/My Drive/Colab Notebooks/test_seg/submission.csv", index=False)

100%|██████████| 69/69 [00:06<00:00, 10.11it/s]
